In [1]:
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer  # use this summarizer for naive metadata summary
#
from milnlp.tokenizers import Tokenizer
from milnlp.collection.collection import Collection

LANGUAGE = "english"
stemmer = Stemmer(LANGUAGE)
summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)
token = Tokenizer(LANGUAGE)
#
term_frequency_threshold = 0.5  
num_sentences = 8  # for composite summary
#
collectionpath = r"C:\Users\zwelz3\Documents\GTRI_Projects\Summarizer\milnlp\Examples"

Using default NLTK tokenizer not english. Custom language tokenizers not available.


In [2]:
from milnlp.collection.collection import get_items

cobj = Collection(collectionpath)
cobj.flist, cobj.dlist = get_items(collectionpath, set(), {collectionpath: 0})


Discovered directory '.ipynb_checkpoints'

Discovered directory 'Archive'

Discovered directory '.ipynb_checkpoints'
-> Discovered file 'example.pdf'
-> Discovered file 'example.txt'
-> Discovered file 'minimum.pdf'
-> Discovered file 'minimum.txt'

Discovered directory 'obj'


## Make new queries

In [3]:
# Placeholder object for containing 'Query List' used to populate GUI
GUI_query_list = []

In [4]:
from milnlp.collection.topic_model import SimpleQuery, ComplexQuery

# Same flag_dict for both simple queries
flag_dict = {
    'case-sensitive': True,
    'whole-word': True,
    'special-delims': False,
    'plurals': False,
    'constituents': False,
}

query = SimpleQuery("OSA")
query.update_flags(flag_dict)  # automatically updates the regex phrase
# query.apply_query(cobj.flist)
GUI_query_list.append(query)
query.__dict__

{'UUID': 'SQ_001',
 'type': 'simple',
 'processed': False,
 'match': None,
 'regex': '(^|(?<=[\\W\x0c]))OSA[\\W]',
 'flags': {'case-sensitive': True,
  'whole-word': True,
  'special-delims': False,
  'plurals': False,
  'constituents': False},
 'phrase': 'OSA'}

In [5]:
query = SimpleQuery("OMS")
query.update_flags(flag_dict)  # automatically updates the regex phrase
# query.apply_query(cobj.flist)
GUI_query_list.append(query)
query.__dict__

{'UUID': 'SQ_002',
 'type': 'simple',
 'processed': False,
 'match': None,
 'regex': '(^|(?<=[\\W\x0c]))OMS[\\W]',
 'flags': {'case-sensitive': True,
  'whole-word': True,
  'special-delims': False,
  'plurals': False,
  'constituents': False},
 'phrase': 'OMS'}

In [6]:
GUI_query_list

In [7]:
import shelve

## Write to shelve
with shelve.open('./obj/query_shelf', 'c') as shelf:
    for query in GUI_query_list:
        print(f"Adding query {query.UUID} to the shelf.")
        shelf[query.UUID] = query

Adding query SQ_001 to the shelf.
Adding query SQ_002 to the shelf.


## Complex Query

In [8]:
shelf_path = './obj/query_shelf'  # default, maybe loaded from GUI
query_list = ['SQ_001', 'SQ_002']  # from combo boxes

cq_example = ComplexQuery(shelf_path, query_list, 'Union')
cq_example.apply_query(cobj.flist, shelf_path=shelf_path)
cq_example.__dict__


Analyzing file:  C:\Users\zwelz3\Documents\GTRI_Projects\Summarizer\milnlp\Examples\minimum.txt
-> Matches found on pages:  {0, 1}

Analyzing file:  C:\Users\zwelz3\Documents\GTRI_Projects\Summarizer\milnlp\Examples\example.txt
-> Matches found on pages:  {0}
Updating simple query SQ_001 on the shelf.

Analyzing file:  C:\Users\zwelz3\Documents\GTRI_Projects\Summarizer\milnlp\Examples\minimum.txt
-> Matches found on pages:  {0, 2}

Analyzing file:  C:\Users\zwelz3\Documents\GTRI_Projects\Summarizer\milnlp\Examples\example.txt
Updating simple query SQ_002 on the shelf.
Updating complex query CQ_001 on the shelf.


{'UUID': 'CQ_001',
 'type': 'complex',
 'processed': True,
 'match': {'C:\\Users\\zwelz3\\Documents\\GTRI_Projects\\Summarizer\\milnlp\\Examples\\minimum.txt': [0,
   1,
   2],
  'C:\\Users\\zwelz3\\Documents\\GTRI_Projects\\Summarizer\\milnlp\\Examples\\example.txt': [0]},
 'phrase': 'SQ_001 or SQ_002',
 'regex': 'union(SQ_001, SQ_002)',
 'operator': 'union',
 'shelf_path': './obj/query_shelf',
 '_dependencies': [<milnlp.collection.topic_model.SimpleQuery at 0x217d16124a8>,
  <milnlp.collection.topic_model.SimpleQuery at 0x217d1612588>]}

In [9]:
shelf_path = './obj/query_shelf'  # default, maybe loaded from GUI
query_list = ['SQ_001', 'SQ_002']  # from combo boxes

cq_example = ComplexQuery(shelf_path, query_list, 'intersection')
cq_example.apply_query(cobj.flist, shelf_path=shelf_path)
cq_example.__dict__

Updating complex query CQ_002 on the shelf.


{'UUID': 'CQ_002',
 'type': 'complex',
 'processed': True,
 'match': {'C:\\Users\\zwelz3\\Documents\\GTRI_Projects\\Summarizer\\milnlp\\Examples\\minimum.txt': [0,
   1,
   2]},
 'phrase': 'SQ_001 and SQ_002',
 'regex': 'intersection(SQ_001, SQ_002)',
 'operator': 'intersection',
 'shelf_path': './obj/query_shelf',
 '_dependencies': [<milnlp.collection.topic_model.SimpleQuery at 0x217d1612da0>,
  <milnlp.collection.topic_model.SimpleQuery at 0x217d1612160>]}